# 第二章 递归状态估计

## Ex2.1 
这道题目与2.4.2 的例题很相似，练习离散贝叶斯滤波的使用。根据题意可知：
$$
\begin{align}
&bel(X_0=broken) &=0.01 \\
&bel(X_0=not\_broken) &= 0.99 \\
&p(Z_t<1|X_t=broken) &= 1 \\
&p(Z_t<1|X_t=not\_broken) &= 1/3
\end{align}
$$
其中$Z_t$ 表示 t 时刻距离传感器的测量值，在 0 ~ 3m 范围内连续取值；$X_t$ 表示距离传感器的状态，取值为离散的 broken ，not_broken。系统初值可以在代码中表示如下

In [1]:
p = [0.01, 0.99]
eta = 1

下面根据贝叶斯滤波算法进行计算，由于这道题中没有对距离传感器进行任何控制，因此贝叶斯滤波的第一步（预测）并不改变系统状态的置信度：

$$
\begin{align}
\bar{bel}(X_t) = bel(X_{t-1})
\end{align}
$$

贝叶斯滤波的第二步（观测），题干中介绍到$Z_t < 1$，于是有以下递推式
$$
\begin{align}
&bel(X_t=broken) &= \eta p(Z_t<1|X_t=broken)\bar{bel}(X_t=broken) \\
&bel(X_t=not\_broken) &= \eta p(Z_t<1|X_t=not\_broken)\bar{bel}(X_t=not\_broken) 
\end{align}
$$

其中 $\eta$ 为归一化因子
$$
\eta = [p(Z_t<1|X_t=broken)\bar{bel}(X_t=broken) + \\ p(Z_t<1|X_t=not\_broken)\bar{bel}(X_t=not\_broken)]^{-1}
$$


对以上5、6、7式代入先验初值1、2进行迭代，即可得到各个时刻距离传感器故障的概率如下

In [2]:
for i in range(1 , 11):
    p = [p[0]*1, p[1]*1/3]
    eta = 1/(p[0] + p[1])
    p = [p[0]*eta, p[1]*eta]
    print(i, p[0], p[1], eta)

(1, 0.029411764705882353, 0.9705882352941176, 2.941176470588235)
(2, 0.08333333333333333, 0.9166666666666666, 2.833333333333333)
(3, 0.2142857142857143, 0.7857142857142857, 2.5714285714285716)
(4, 0.45000000000000007, 0.55, 2.1)
(5, 0.7105263157894737, 0.2894736842105263, 1.5789473684210524)
(6, 0.8804347826086958, 0.11956521739130438, 1.2391304347826089)
(7, 0.9566929133858267, 0.043307086614173235, 1.0866141732283463)
(8, 0.9851351351351351, 0.014864864864864866, 1.0297297297297296)
(9, 0.9949954504094631, 0.0050045495905368526, 1.0100090991810737)
(10, 0.9983262325015215, 0.0016737674984783934, 1.0033475349969567)


最后推导公式：
$$
\begin{align}
\begin{array}
\ bel(X_t=broken) &= {\eta}_t\times1\times bel(X_{t-1}=broken) \\
&= {\eta}_t{\eta}_{t-1}\times1\times bel(X_{t-2}=broken) \\
&=\prod_{i=0}^n {\eta}_i bel(X_0=broken)
\end{array}
\\
\begin{array}
\ bel(X_t=not\_broken) &= {\eta}_t\times \frac{1}{3} \times bel(X_{t-1}=not\_broken) \\
&= {\eta}_t{\eta}_{t-1}\times \frac{1}{3} \times bel(X_{t-2}=broken) \\
&=\prod_{i=0}^n {\eta}_i \times \frac{1}{3^n} \times bel(X_0=not\_broken)
\end{array}
\end{align}
$$
其中 $\prod_{i=0}^n {\eta}_i$ 满足归一化要求，即
$$
\prod_{i=0}^n {\eta}_i = [bel(X_0=broken) + \frac{1}{3^n} \times bel(X_0=not\_broken)]^{-1}
$$
因此，传感器失效的概率为
$$
bel(X_t=broken) = \frac{bel(X_0=broken)}{bel(X_0=broken) + \frac{1}{3^n} \times bel(X_0=not\_broken)}
$$

## 2. 已知天气状态转移矩阵

|     标题     | 晴（明天） | 多云 | 雨   |
| ------------ | ---------- | ---- | ---- |
| 晴（今天）   | 0.8        | 0.2  | 0    |
| 多云（今天） | 0.4        | 0.4  | 0.2  |
| 雨（今天）   | 0.2        | 0.6  | 0.2  |

### a) 第一天是晴天，求接下来三天是多云、多云、雨的概率

可以通过贝叶斯公式得到
$$
\begin{align}
\begin{array}
P(X_4=rainy) &= P(X_4=rainy|X_3=cloudy)P(X_3=cloudy) \\
&= P(X_4=rainy|X_3=cloudy)P(X_3=cloudy|X_2=cloudy)\\
& P(X_2=cloudy|X_1=sunny)P(X_1=sunny) \\
&= 0.2 \times 0.4 \times 0.2 \\
&= 0.016
\end{array}
\end{align}
$$

### b)编程随机产生天气序列的仿真器

In [3]:
import random as rd

cnt_weath = [0,0,0]
weath = ["sunny", "cloudy", "windy"]
cnt = 0
trans = [[0.8, 1.0, 1.0],[0.4, 0.8,1.0],[0.2,0.8,1.0]]

yest = 0
today = 0
while cnt < 10000000:
	tran = trans[yest]
	p = rd.random()
	if p < tran[0]:
		today = 0
	elif p < tran[1]:
		today = 1
	else:
		today = 2
	# print(weath[today])
	cnt_weath[today] = cnt_weath[today] + 1
	cnt = cnt+1
	yest = today

print(float(cnt_weath[0])/cnt, float(cnt_weath[1])/cnt, float(cnt_weath[2])/cnt)

(0.6427326, 0.2858765, 0.0713909)


### c)计算平稳分布

首先需要知道平稳分布的定义，根据[维基百科](<https://en.wikipedia.org/wiki/Markov_chain#Steady-state_analysis_and_limiting_distributions>)的介绍，平稳分布是指不随时间变化的状态分布，即
$$
\lim\limits_{n \to \infty} {X_n}=\lim\limits_{n \to \infty}{A^nX_0}
$$
使用b) 中设计的随机天气序列仿真器，在天气序列足够长时，天气出现的频率就是平稳分布的概率，计算得到
$$
P_{static}=[0.6430726, 0.2855164, 0.071411]^T
$$
### d)通过公式计算平稳分布

根据平稳分布的定义，我们需要求的其实是状态转移矩阵的特征值1对应的特征向量
$$
Ax=x
$$
这里我们采用偷懒的方法，使用[矩阵特征向量在线计算工具](<https://zs.symbolab.com/solver/matrix-eigenvectors-calculator>)，计算得到特征向量为
$$
[9, 4, 1]^T
$$
对该向量进行概率归一化得到
$$
P_{static}=[9/14, 2/7, 1/14]^T
$$

### f) 计算给定今天天气时，昨天天气的概率表

需要计算以下条件概率
$$
P(X_{t-1}=i|X_t=j)=\frac{P(X_{t}=j|X_{t-1}=i)P(X_{t-1}=i)}{P(X_{t}=j)} \\
P(X_{t}=j)=\sum_{i}^3 P(X_{t}=j|X_{t-1}=i)P(X_{t-1}=i)
$$
其中$$P(X_{t-1}=i)$$作为先验概率，由平稳分布给出，从而计算得出概率表

|   标题     | 晴（今天） | 多云 | 雨   |
| ---------- | ---------- | ---- | ---- |
| 晴（昨天） | 0.8        | 0.45 | 0    |
| 多云       | $$8/45$$   | 0.4  | 0.8  |
| 雨         | $$1/45$$   | 0.15 | 0.2  |



### g) 如果天气状态转移概率与季节相关，这个过程还满足马尔科夫特性吗

不满足。马尔科夫特性要求，当前时刻的系统状态只与上一时刻的系统状态有关，与其他变量无关。如果引入季节这一变量，就会破坏马尔科夫特性。但是在一个季节内，状态转移矩阵不发生变化，该过程依然保持马尔科夫特性。

## 3. 若有传感器观测矩阵

|     标题     | 晴（实际） | 多云（实际） | 雨（实际） |
| ------------ | ---------- | ------------ | ---------- |
| 晴（观测）   | 0.6        | 0.3          | 0          |
| 多云（观测） | 0.4        | 0.7          | 0          |
| 雨（观测）   | 0          | 0            | 1          |

### a) 已知第一天是晴天，2~5天观测到为多云，多云，雨，晴，求第五天是晴天的概率

由第二题可知状态转移矩阵为
$$
A=\begin{bmatrix}
0.8 &0.4 &0.2 \\
0.2 &0.4 &0.6 \\
0   &0.2 &0.2
\end{bmatrix}
$$
由本题题干可知观测矩阵为
$$
C=\begin{bmatrix}
0.6 &0.3 &0 \\
0.4 &0.7 &0 \\
0   &0   &1
\end{bmatrix}
$$
根据贝叶斯准则
$$
\begin{align}
\bar{bel}(x_t|x_{t-1})&=A\cdot bel(x_{t-1}) \\
bel(X_t=i|Z_t=j)&=\eta P(Z_t=j|X_t=i)P(X_t=i) \\
\eta &= (\sum_{i=1}^n{P(Z_t=j|X_t=i)P(X_t=i)})^{-1}
\end{align}
$$
初值和观测序列
$$
bel(x_1)=\begin{bmatrix}1 &0 &0 \end{bmatrix}^T \\
z_2=cloudy, z_3=cloudy, z_4=rainy, z_5=sunny
$$
迭代计算可得

|  标题  | $z_t$  | $$\bar{bel}(x_t)$$  | $bel(x_t)$                               | $$\eta$$     |
| ---- | ------ | ------------------------------------------- | ------------------------------------------- | ------------ |
| 1    |   sunny     | $$\begin{bmatrix}1 &0 &0 \end{bmatrix}^T$$                | $$\begin{bmatrix}1 &0 &0 \end{bmatrix}^T$$             | 1            |
| 2    | cloudy | $$\begin{bmatrix}0.8 &0.2 &0 \end{bmatrix}^T$$            | $$\begin{bmatrix}16/23 &7/23 &0 \end{bmatrix}^T$$      | $$50/23$$    |
| 3    | cloudy | $$\begin{bmatrix}78/115 &30/115 &7/115 \end{bmatrix}^T$$  | $$\begin{bmatrix}312/522 &210/522 &0 \end{bmatrix}^T$$ | $$1150/522$$ |
| 4    | rainy  | $$\begin{bmatrix}0.6390 &0.2805 &0.0805 \end{bmatrix}^T$$ | $$\begin{bmatrix}0 &0 &1 \end{bmatrix}^T$$             | 124.2857     |
| 5    | sunny  | $$\begin{bmatrix}0.2 &0.6 &0.2 \end{bmatrix}^T$$          | $$\begin{bmatrix}0.4 &0.6 &0.0 \end{bmatrix}^T$$       | 3.3333       |

由上面结果可得，第五天是晴天的概率为0.4

### b) 当2~4天的观测序列为晴，晴，雨时，用两种方式讨论最有可能的天气序列：1）考虑当天以及之前的观测结果；2）统一考虑所有天气

根据a)中的方法，当观测序列为晴、晴、雨时，最有可能的天气序列为

| 日期 | 天气 | 概率   |
| ---- | ---- | ------ |
| 1    | 晴   | 1.0    |
| 2    | 晴   | 0.8889 |
| 3    | 晴   | 0.8718 |
| 4    | 雨   | 1.0    |

统一考虑所有天气时
$$
\begin{align}
\begin{array}
/P(X_2=i|z_{2:4})&=\eta P(Z_2=sunny|X_2=i,z_{3:4})P(X_2=i|z_{3:4}) \\
&=\eta P(Z_2=sunny|X_2=i)P(X_2=i|z_{3:4}) \\
&=\eta P(Z_2=sunny|X_2=i)P(Z_3=sunny|X_2=i,z_4)P(X_2=i|z_4) \\
&=\eta P(Z_2=sunny|X_2=i)P(Z_3=sunny|X_2=i) \\
&\times P(Z_4=rainy|X_2=i)P(X_2=i|z_2)
\end{array}
\end{align}
$$
其中
$$
\begin{align}
\begin{array}
/P(Z_3=sunny|X_2=i)&=\sum_{j=1}^n{P(Z_3=sunny|X_3=j,X_2=i)P(X_3=j|X_2=i)} \\
&=(CA)_{1,i}
\end{array}
\\
\begin{array}
/P(Z_4=rainy|X_2=i)&=\sum_{j=1}^n{P(Z_4=rainy|X_4=j,X_2=i)P(X_4=j|X_2=i)} \\
&=\sum_{j=1}^n{P(Z_4=rainy|X_4=j)\sum_{k=1}^nP(X_4=j|X_3=k)P(X_3=k|X_2=i)} \\
&=(CAA)_{3,i}
\end{array}
\end{align}
$$
同理可得第3天和第4天的概率
$$
\begin{align}
\begin{array}
/P(X_3=i|z_{2:4})&=P(Z_4=rainy|X_3=i)P(X_3=i|z_{2:3}) \\
&=(CA)_{3,i}P(X_3=i|z_{2:3}) \\
\end{array}
\end{align}
$$
其中$$P(X_3=i|z_{2:3})$$和$$P(X_4=i|z_{2:4})$$已经在前一小问中求得，汇总得到2~4天的天气概率

| 日期 | 天气 | 概率 |
| ---- | ---- | ---- |
| 1    | 晴   | 1    |
| 2    | 晴   | 0.8  |
| 3    | 多云 | 1    |
| 4    | 雨   | 1    |



### c) 若第2~4天的天气测量分别为晴，晴，雨，那么最有可能的天气序列是什么，该天气序列的概率是多少

根据b)的解答，最可能的天气序列为晴、多云、雨，该序列概率为每天概率的乘积，为0.8.

我们会发现，综合考虑全序列的传感器数据，和只考虑当天的观测数据，得到的天气序列的预测是不同的。根据状态转移矩阵，不存在由晴天变成雨天的转换，而第三天为雨天的概率为1，所以虽然第三天传感器检测为晴天，但综合考虑第四天的概率后，第三天的天气应为雨天。这道题使用简单的对比，向我们形象地说明：如果采用全时间序列的观测信息，能够得到更加准确的后验概率估计。

## 4. 考虑一维高斯分布，初始位置$x_{init}=1000m$，方差$\sigma_{init}^2=900m^2$，观测位置$x_{GPS}=1100m$,方差$\sigma_{GPS}^2=100m^2$

### a) 写出先验$p(x)$和观测$p(z|x)$的概率密度函数

根据高斯分布公式可得
$$
p(x)={1}/{\sqrt{2\pi\sigma_{init}^2}}\cdot e^{-\frac{(x-x_{init})^2}{2\sigma_{init}^2}} \\

p(z|x)={1}/{\sqrt{2\pi\sigma_{GPS}^2}}\cdot e^{-\frac{(z-x)^2}{2\sigma_{GPS}^2}}
$$

### b) 使用贝叶斯准则，后验概率$p(x|z)$是多少，能证明该分布是高斯吗

根据贝叶斯准则
$$
p(x|z)=\frac{p(z|x)p(x)}{p(z)}
$$
其中
$$
\begin{align}
\begin{array}
/p(z)&=\int_{-\infty}^{+\infty} p(z|x)p(x)dx \\
&=\int_{-\infty}^{+\infty}{1}/{\sqrt{2\pi\sigma_{init}^2}}\cdot e^{-\frac{(x-x_{init})^2}{2\sigma_{init}^2}} \cdot {1}/{\sqrt{2\pi\sigma_{GPS}^2}}\cdot e^{-\frac{(z-x)^2}{2\sigma_{GPS}^2}}dx \\
&=\int_{-\infty}^{+\infty}\frac{1}{2\pi \sigma_{init}\sigma_{GPS}}\cdot e^{-\frac{(x-x_{init})^2}{2\sigma_{init}^2}-\frac{(z-x)^2}{2\sigma_{GPS}^2}}dx
\end{array}
\end{align}
$$
对指数二项式进行分析
$$
\begin{align}
\begin{array}
\frac{(x-x_{init})^2}{2\sigma_{init}^2}+\frac{(z-x)^2}{2\sigma_{GPS}^2}
=&\frac{x^2-2xx_{init}+x_{init}^2}{18\sigma_{GPS}^2}+\frac{z^2 -2xz +x^2}{2\sigma_{GPS}^2} \\
=&\frac{10(x^2-2x(\frac{x_{init}+9z}{10})+(\frac{x_init+9z}{10})^2)}{18\sigma_{GPS}^2}+\frac{9z^2+x_{init}^2-\frac{(x_init+9z)^2}{10}}{18\sigma_{GPS}^2}
\end{array}
\end{align}
$$
第一项是x的函数，可以写成正态分布的形式，第二项与x无关，可从积分式内提取出来，计算得到
$$
p(z)=\frac{1}{2\pi \sigma_{init}\sigma_{GPS}}\cdot e^{-\frac{9z^2+x_{init}^2-\frac{(x_{init}+9z)^2}{10}}{18\sigma_{GPS}^2}} \cdot \sqrt{2\pi}\sqrt{9/10}\sigma_{GPS} \\
=\frac{1}{\sqrt{2\pi} \sqrt{10}\sigma_{GPS}}\cdot e^{-\frac{9z^2+x_{init}^2-\frac{(x_{init}+9z)^2}{10}}{18\sigma_{GPS}^2}}
$$
那么后验的分布为
$$
p(z|x)=\frac{\frac{1}{2\pi \sigma_{init}\sigma_{GPS}}\cdot e^{-\frac{10(x^2-2x(\frac{x_{init}+9z}{10})+(\frac{x_init+9z}{10})^2)}{18\sigma_{GPS}^2}-\frac{9z^2+x_{init}^2-\frac{(x_init+9z)^2}{10}}{18\sigma_{GPS}^2}}}{\frac{1}{\sqrt{2\pi} \sqrt{10}\sigma_{GPS}}\cdot e^{-\frac{9z^2+x_{init}^2-\frac{(x_{init}+9z)^2}{10}}{18\sigma_{GPS}^2}}} \\
=\frac{1}{\sqrt{2\pi}\sqrt{9/10} \sigma_{GPS}}e^{-\frac{(x-\frac{9z+x_{init}}{10})^2}{2\times9/10\sigma_{GPS}^2}}
$$
是满足正态分布$$p(x|z)\sim N(\frac{9z+x_{init}}{10}, 9/10\sigma_{GPS}^2)$$



### c) 测量$x_{GPS}=1100m$怎样得出先验和GPS接收器的误差概率信息

不是很理解这道题目需要求什么，这里大胆猜测一下，是对GPS接收器进行误差分析，问的是怎样获取GPS接收器的值和方差，GPS的先验数据经过滤波得到，误差信息通过对时间统计得到。

## 5. 由式（2.17）推导（2.18）和（2.19），以及本书叙述的概率准则

根据式（2.17）$$p(x,y|z)=p(x|z)p(y|z)$$可得
$$
\begin{align}
p(x|z)&=\frac{p(x,y|z)}{p(y|z)}=\frac{p(x|z)p(y|z)}{p(y|z)}=p(x|y,z) \\
p(y|z)&=\frac{p(x,y|z)}{p(x|z)}=p(y|x,z)
\end{align}
$$
根据一阶马尔科夫假设，当前观测只与系统当前状态$$x_t$$相关，与其他变量相互独立，于是可以得到
$$
p(z_i|x_{0:t}, z_{1:t-1},u_{1:t-1})=p(z_i|x_t)
$$

## 6. 证明式（2.25），这个等式的意义是什么

根据期望的性质
$$
\begin{align}
\begin{array}
/Cov[X]&=E[X-E[X]]^2\\
&=E[X^2-2X\cdot E[X]+(E[X])^2]\\
&=E[X^2]-2E[X]E[X]+(E[X])^2\\
&=E[X^2]-(E[X])^2
\end{array}
\end{align}
$$
这说明方差是协方差的特殊形式，对于多元分布而言，协方差是一个方阵。

# 小结

- 1,2,3题都是在使用贝叶斯准则解决离散问题，其中第三题给了另外一种不同的应用方法，不仅仅是按照时序计算的贝叶斯滤波，还可以应用在解决全概率问题
- 4题解决的是一维连续问题，推导了高斯分布下的贝叶斯准则（即卡尔曼滤波），证明了后验概率也是高斯分布，更深入地理解了卡尔曼滤波
- 5题帮助更好的理解一阶马尔科夫假设下的贝叶斯准则